In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import scipy, os
from sklearn.metrics import confusion_matrix
import seaborn as sns

import sys
sys.path.append('../../Share/')
sys.path.append('../../Share/Manual_processing/')
import baseline, config, Model, utils, Same_with_MATLAB, Feature_info

import warnings
warnings.filterwarnings('ignore')


def restore_labels(mat, labels_windowed):

    original_length = mat['Data_ADC'].shape[1]
    win_size = 1000 # 200ms 10
    win_step = 50
    #valid_length = original_length - 2 * 60

    label_full = np.zeros(original_length, dtype=labels_windowed.dtype) # 복원될 시계열 레이블 (원본 길이)

    # 슬라이딩 윈도우 인덱스 따라 레이블 채워넣기
    for i, label in enumerate(labels_windowed):
        start = 60 + i * win_step
        end = start + win_size
        if end <= original_length - 60:
            label_full[start:end] = label

    return label_full


def filtering_zero(X, y, erase_label):
    # 1. erase_label 제거
    keep_indices = y != erase_label
    X = X[keep_indices]
    y = y[keep_indices]

    # 2. erase_label보다 큰 값은 1씩 감소
    y = np.where(y > erase_label, y - 1, y)

    return X, y

def vis_graph(history):
    plt.figure(figsize=(8, 4))
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.legend()
    plt.show()

def heatmap_confusion_matrix(X_test, y_test, model):
    # Predict class labels on the test set
    y_pred_probs = model.predict(X_test, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)  # convert softmax probs to predicted class
    y_true = y_test
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(np.max(y_test)+1), yticklabels=range(np.max(y_test)+1))
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()


from collections import Counter

def balance_data(X, y):
    # Count samples per class
    class_counts = Counter(y)
    min_count = min(class_counts.values())  # target: balance all to minority count

    indices_list = []

    for label in sorted(class_counts.keys()):
        label_indices = np.where(y == label)[0]
        selected_indices = np.random.choice(label_indices, size=min_count, replace=False)
        indices_list.extend(selected_indices)

    # Shuffle all selected indices
    balanced_indices = np.random.permutation(indices_list)

    # Subset the data
    X_balanced = X[balanced_indices]
    y_balanced = y[balanced_indices]

    return X_balanced, y_balanced

In [2]:
def return_X_y(path, balance):
    fs, lower_cutoff, upper_cutoff = Feature_info.fs, Feature_info.lower_cutoff, Feature_info.upper_cutoff
    # fs, lower_cutoff, upper_cutoff = Feature_info.fs, 1, 300
    filter_b, filter_a = Same_with_MATLAB.cheby2(4, 30, [lower_cutoff / (fs/2), upper_cutoff / (fs/2)], btype='bandpass')

    data_per_class_files = os.listdir(path)
    X, y = [], []

    for cls in data_per_class_files:
        input_path = path+cls+'/'
        files = os.listdir(input_path)
        mat = scipy.io.loadmat(input_path+files[0])
        label = mat['Data_Cls'].reshape(-1)  # shape: (1, 1729)

        feat_mean = np.tile(Feature_info.feat_mean_lst, (4, 1))
        feat_std = np.tile(Feature_info.feat_std_lst, (4, 1))

        mapped_label = np.where(label == 0, 0, int(cls))
        restored_label = restore_labels(mat, mapped_label)

        #print(mat['Data_ADC'].shape, mat['Data_Cls'].shape, restored_label.shape)
        extractor = Same_with_MATLAB.EMGFeatureExtractor(feat_mean, feat_std, filter_b, filter_a, Norm_bool=False, num_feature_set=14) #I tried 23, but not so good
        extractor.buffer = mat['Data_ADC']
        #1000, 50 = winsize and winstep
        features, labels = extractor.extract_features_with_labels(win_size=1000, win_step=50, feat_exclude=25, filtering=False, restored_label=restored_label)

        features = np.transpose(features, (2, 0, 1))  # shape: (1729, 4, 14)
        X.append(features)
        y.append(labels)
        #print(features.shape, labels.shape)

    X_train = np.concatenate(X, axis=0)
    y_train = np.concatenate(y, axis=0)
    X_train = X_train[:, :, :, np.newaxis]
    #print(pd.Series(y_train).value_counts())
    print(X_train.shape, y_train.shape)

    if balance:
        X_train, y_train = balance_data(X_train, y_train)

    return X_train, y_train


def train_model_feature_wise(X_train, y_train, X_test, y_test, num_channels):
    X_train, y_train = filtering_zero(X_train, y_train, erase_label=0)
    X_test, y_test = filtering_zero(X_test, y_test, erase_label=0)

    feature_idx = 2
    One_X_train = X_train[:, num_channels, feature_idx, :]
    One_X_test = X_test[:, num_channels, feature_idx, :]
    print(One_X_train.shape, One_X_test.shape)

    model = Model.Original_model_1DCNN(One_X_train.shape[1:], num_class=np.max(y_train)+1)

    history, _ = Model.Train_model(model, One_X_train, y_train, One_X_test, y_test,
                                    set_epoch=200, set_batch_size=64, Model_name='V0', set_verbose=False, save_model_set=False)

    return np.max(history.history['val_accuracy'])


def run(subject, modality, num_channels):
    if modality == 'EMG': bluetooth_id = 'E8DD80E550BB'
    elif modality == 'ENG': bluetooth_id = 'E9AD0E7DCC2B'
    else:
        print("Modality should be EMG or ENG")
        return

    X_train, y_train = return_X_y(path = base_path+f'{subject}/{modality}_v1/{bluetooth_id}/raw/', balance=True)
    X_test, y_test = return_X_y(path = base_path+f'{subject}/{modality}_v2/{bluetooth_id}/raw/', balance=True)
    result1 = train_model_feature_wise(X_train, y_train, X_test, y_test, num_channels)

    X_train, y_train = return_X_y(path = base_path+f'{subject}/{modality}_v2/{bluetooth_id}/raw/', balance=True)
    X_test, y_test = return_X_y(path = base_path+f'{subject}/{modality}_v1/{bluetooth_id}/raw/', balance=True)
    result2 = train_model_feature_wise(X_train, y_train, X_test, y_test, num_channels)

    return np.mean([result1, result2])

In [3]:
base_path = 'C:/Users/hml76/PycharmProjects/MindForce/data/EMG_ENG/'
acc_all_ch4, acc_all_ch3, acc_all_ch2, acc_all_ch1 = [], [], [], []

In [4]:
SUB = 'Hunmin'
result = run(subject=SUB, modality='ENG', num_channels=[0,1,2,3])
acc_all_ch4.append(result)

idx = np.random.choice(4, size=3, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch3.append(result)

idx = np.random.choice(4, size=2, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch2.append(result)

idx = np.random.choice(4, size=1, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch1.append(result)

(33864, 4, 14, 1) (33864,)
(33609, 4, 14, 1) (33609,)
(1976, 4, 1) (1112, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 15.79%
Maximum validation accuracy : 16.55%
(33609, 4, 14, 1) (33609,)
(33864, 4, 14, 1) (33864,)
(1112, 4, 1) (1976, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 18.26%
Maximum validation accuracy : 15.13%
(33864, 4, 14, 1) (33864,)
(33609, 4, 14, 1) (33609,)
(1976, 3, 1) (1112, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 17.05%
Maximum validation accuracy : 19.6%
(33609, 4, 14, 1) (33609,)
(33864, 4, 14, 1) (33864,)
(1112, 3, 1) (1976, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 16.82%
Maximum validation accuracy : 14.73%
(33864, 4, 14, 1) (33864,)
(33609, 4, 14, 1) (33609,)
(1976, 2, 1) (1112, 2, 1)
Start

# Byeongchan

In [5]:
SUB = 'Byeongchan'
result = run(subject=SUB, modality='ENG', num_channels=[0,1,2,3])
acc_all_ch4.append(result)

idx = np.random.choice(4, size=3, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch3.append(result)

idx = np.random.choice(4, size=2, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch2.append(result)

idx = np.random.choice(4, size=1, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch1.append(result)

(21215, 4, 14, 1) (21215,)
(21285, 4, 14, 1) (21285,)
(1200, 4, 1) (1140, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 35.5%
Maximum validation accuracy : 29.91%
(21285, 4, 14, 1) (21285,)
(21215, 4, 14, 1) (21215,)
(1140, 4, 1) (1200, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 32.54%
Maximum validation accuracy : 30.83%
(21215, 4, 14, 1) (21215,)
(21285, 4, 14, 1) (21285,)
(1200, 3, 1) (1140, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 38.42%
Maximum validation accuracy : 30.61%
(21285, 4, 14, 1) (21285,)
(21215, 4, 14, 1) (21215,)
(1140, 3, 1) (1200, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 28.42%
Maximum validation accuracy : 29.83%
(21215, 4, 14, 1) (21215,)
(21285, 4, 14, 1) (21285,)
(1200, 2, 1) (1140, 2, 1)
Start

In [6]:
SUB = 'Hongjun'
result = run(subject=SUB, modality='ENG', num_channels=[0,1,2,3])
acc_all_ch4.append(result)

idx = np.random.choice(4, size=3, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch3.append(result)

idx = np.random.choice(4, size=2, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch2.append(result)

idx = np.random.choice(4, size=1, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch1.append(result)

(21265, 4, 14, 1) (21265,)
(21306, 4, 14, 1) (21306,)
(1820, 4, 1) (1490, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 47.64%
Maximum validation accuracy : 34.97%
(21306, 4, 14, 1) (21306,)
(21265, 4, 14, 1) (21265,)
(1490, 4, 1) (1820, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 40.13%
Maximum validation accuracy : 42.2%
(21265, 4, 14, 1) (21265,)
(21306, 4, 14, 1) (21306,)
(1820, 3, 1) (1490, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 46.87%
Maximum validation accuracy : 30.27%
(21306, 4, 14, 1) (21306,)
(21265, 4, 14, 1) (21265,)
(1490, 3, 1) (1820, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 34.23%
Maximum validation accuracy : 29.29%
(21265, 4, 14, 1) (21265,)
(21306, 4, 14, 1) (21306,)
(1820, 2, 1) (1490, 2, 1)
Start

In [7]:
SUB = 'Jongin'
result = run(subject=SUB, modality='ENG', num_channels=[0,1,2,3])
acc_all_ch4.append(result)

idx = np.random.choice(4, size=3, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch3.append(result)

idx = np.random.choice(4, size=2, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch2.append(result)

idx = np.random.choice(4, size=1, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch1.append(result)

(21275, 4, 14, 1) (21275,)
(21286, 4, 14, 1) (21286,)
(2630, 4, 1) (2720, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 39.16%
Maximum validation accuracy : 43.82%
(21286, 4, 14, 1) (21286,)
(21275, 4, 14, 1) (21275,)
(2720, 4, 1) (2630, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 42.1%
Maximum validation accuracy : 38.44%
(21275, 4, 14, 1) (21275,)
(21286, 4, 14, 1) (21286,)
(2630, 3, 1) (2720, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 35.44%
Maximum validation accuracy : 41.58%
(21286, 4, 14, 1) (21286,)
(21275, 4, 14, 1) (21275,)
(2720, 3, 1) (2630, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 42.13%
Maximum validation accuracy : 40.34%
(21275, 4, 14, 1) (21275,)
(21286, 4, 14, 1) (21286,)
(2630, 2, 1) (2720, 2, 1)
Start

In [8]:
SUB = 'Youngchul'
result = run(subject=SUB, modality='ENG', num_channels=[0,1,2,3])
acc_all_ch4.append(result)

idx = np.random.choice(4, size=3, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch3.append(result)

idx = np.random.choice(4, size=2, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch2.append(result)

idx = np.random.choice(4, size=1, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch1.append(result)

(34068, 4, 14, 1) (34068,)
(34016, 4, 14, 1) (34016,)
(2336, 4, 1) (2168, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 28.6%
Maximum validation accuracy : 17.2%
(34016, 4, 14, 1) (34016,)
(34068, 4, 14, 1) (34068,)
(2168, 4, 1) (2336, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 28.14%
Maximum validation accuracy : 14.51%
(34068, 4, 14, 1) (34068,)
(34016, 4, 14, 1) (34016,)
(2336, 3, 1) (2168, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 27.05%
Maximum validation accuracy : 13.33%
(34016, 4, 14, 1) (34016,)
(34068, 4, 14, 1) (34068,)
(2168, 3, 1) (2336, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 26.8%
Maximum validation accuracy : 19.65%
(34068, 4, 14, 1) (34068,)
(34016, 4, 14, 1) (34016,)
(2336, 2, 1) (2168, 2, 1)
Start T

In [9]:
SUB = 'Harold'
result = run(subject=SUB, modality='ENG', num_channels=[0,1,2,3])
acc_all_ch4.append(result)

idx = np.random.choice(4, size=3, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch3.append(result)

idx = np.random.choice(4, size=2, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch2.append(result)

idx = np.random.choice(4, size=1, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch1.append(result)

(33852, 4, 14, 1) (33852,)
(34016, 4, 14, 1) (34016,)
(2680, 4, 1) (2752, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 31.53%
Maximum validation accuracy : 21.69%
(34016, 4, 14, 1) (34016,)
(33852, 4, 14, 1) (33852,)
(2752, 4, 1) (2680, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 28.78%
Maximum validation accuracy : 16.38%
(33852, 4, 14, 1) (33852,)
(34016, 4, 14, 1) (34016,)
(2680, 3, 1) (2752, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 23.69%
Maximum validation accuracy : 20.49%
(34016, 4, 14, 1) (34016,)
(33852, 4, 14, 1) (33852,)
(2752, 3, 1) (2680, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 27.03%
Maximum validation accuracy : 19.89%
(33852, 4, 14, 1) (33852,)
(34016, 4, 14, 1) (34016,)
(2680, 2, 1) (2752, 2, 1)
Star

In [10]:
SUB = 'Carlson'
result = run(subject=SUB, modality='ENG', num_channels=[0,1,2,3])
acc_all_ch4.append(result)

idx = np.random.choice(4, size=3, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch3.append(result)

idx = np.random.choice(4, size=2, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch2.append(result)

idx = np.random.choice(4, size=1, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch1.append(result)

(21255, 4, 14, 1) (21255,)
(21235, 4, 14, 1) (21235,)
(1660, 4, 1) (2275, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 45.9%
Maximum validation accuracy : 27.38%
(21235, 4, 14, 1) (21235,)
(21255, 4, 14, 1) (21255,)
(2275, 4, 1) (1660, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 40.18%
Maximum validation accuracy : 29.4%
(21255, 4, 14, 1) (21255,)
(21235, 4, 14, 1) (21235,)
(1660, 3, 1) (2275, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 43.43%
Maximum validation accuracy : 27.78%
(21235, 4, 14, 1) (21235,)
(21255, 4, 14, 1) (21255,)
(2275, 3, 1) (1660, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 36.57%
Maximum validation accuracy : 34.22%
(21255, 4, 14, 1) (21255,)
(21235, 4, 14, 1) (21235,)
(1660, 2, 1) (2275, 2, 1)
Start 

In [11]:
SUB = 'Minjeong'
result = run(subject=SUB, modality='ENG', num_channels=[0,1,2,3])
acc_all_ch4.append(result)

idx = np.random.choice(4, size=3, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch3.append(result)

idx = np.random.choice(4, size=2, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch2.append(result)

idx = np.random.choice(4, size=1, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch1.append(result)

(33926, 4, 14, 1) (33926,)
(33915, 4, 14, 1) (33915,)
(2080, 4, 1) (2192, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 26.15%
Maximum validation accuracy : 13.23%
(33915, 4, 14, 1) (33915,)
(33926, 4, 14, 1) (33926,)
(2192, 4, 1) (2080, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 14.78%
Maximum validation accuracy : 13.61%
(33926, 4, 14, 1) (33926,)
(33915, 4, 14, 1) (33915,)
(2080, 3, 1) (2192, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 27.07%
Maximum validation accuracy : 16.7%
(33915, 4, 14, 1) (33915,)
(33926, 4, 14, 1) (33926,)
(2192, 3, 1) (2080, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 15.05%
Maximum validation accuracy : 15.58%
(33926, 4, 14, 1) (33926,)
(33915, 4, 14, 1) (33915,)
(2080, 2, 1) (2192, 2, 1)
Start

In [12]:
SUB = 'Xianyu'
result = run(subject=SUB, modality='ENG', num_channels=[0,1,2,3])
acc_all_ch4.append(result)

idx = np.random.choice(4, size=3, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch3.append(result)

idx = np.random.choice(4, size=2, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch2.append(result)

idx = np.random.choice(4, size=1, replace=False)
result = run(subject=SUB, modality='ENG', num_channels=idx)
acc_all_ch1.append(result)

(33865, 4, 14, 1) (33865,)
(33896, 4, 14, 1) (33896,)
(1112, 4, 1) (888, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 23.56%
Maximum validation accuracy : 14.3%
(33896, 4, 14, 1) (33896,)
(33865, 4, 14, 1) (33865,)
(888, 4, 1) (1112, 4, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 23.09%
Maximum validation accuracy : 17.0%
(33865, 4, 14, 1) (33865,)
(33896, 4, 14, 1) (33896,)
(1112, 3, 1) (888, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 20.5%
Maximum validation accuracy : 14.19%
(33896, 4, 14, 1) (33896,)
(33865, 4, 14, 1) (33865,)
(888, 3, 1) (1112, 3, 1)
Start Training (total epochs: 200)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 27.7%
Maximum validation accuracy : 17.0%
(33865, 4, 14, 1) (33865,)
(33896, 4, 14, 1) (33896,)
(1112, 2, 1) (888, 2, 1)
Start Training

In [17]:
std_values = [np.std(acc_all_ch4), np.std(acc_all_ch3), np.std(acc_all_ch2), np.std(acc_all_ch1)]
std_clean_values = [float(v) for v in std_values]
print(std_clean_values)

[0.08343234445005425, 0.08492193274795812, 0.09487073400413076, 0.05375169177908187]


In [14]:
values = [np.mean(acc_all_ch4), np.mean(acc_all_ch3), np.mean(acc_all_ch2), np.mean(acc_all_ch1)]
clean_values = [float(v)*100 for v in values]
print(clean_values)

[24.253310842646492, 24.17076826095581, 23.127614541186226, 20.272833274470436]


In [15]:
#with norm / no filtering / 1000 windows size / 50 inc
acc_all_ch4 = [0.17922443, 0.26614036, 0.34922376, 0.41592973, 0.17068948, 0.21390739, 0.30677943, 0.20652547, 0.16426048]
acc_all_ch3 = [0.15155972, 0.26872806, 0.32417583, 0.40810572, 0.15533697, 0.20095616, 0.23474248, 0.13834573, 0.18732581]
acc_all_ch2 = [0.14669924, 0.27037281, 0.33773692, 0.39835536, 0.18833721, 0.22466212, 0.36449291, 0.14564501, 0.14097235]
acc_all_ch1 = [0.13520519, 0.23039474, 0.30683494, 0.23633485, 0.187733, 0.17585908, 0.26064743, 0.13920901, 0.18112402]

values = [np.mean(acc_all_ch1), np.mean(acc_all_ch2), np.mean(acc_all_ch3), np.mean(acc_all_ch4)]
clean_values = [float(v)*100 for v in values]
print(clean_values)

[20.592691777777773, 24.636377, 22.99196088888889, 25.25200588888889]


In [19]:
np.array(acc_all_ch2)

array([0.14669924, 0.27037281, 0.33773692, 0.39835536, 0.18833721,
       0.22466212, 0.36449291, 0.14564501, 0.14097235])